In [58]:
import gurobipy as gp
import pandas as pd
from gurobipy import GRB


# DATA SETUP
path = 'https://raw.githubusercontent.com/Gurobi/modeling-examples/master/optimization101/Modeling_Session_1/'
transp_cost = pd.read_csv(path + 'cost.csv')
# get production and distribution locations from data frame
production = list(transp_cost['production'].unique())
distribution = list(transp_cost['distribution'].unique())
transp_cost = transp_cost.set_index(['production','distribution']).squeeze()

max_prod = pd.Series([180,200,140,80,180], index = production, name = "max_production")
n_demand = pd.Series([89,95,121,101,116,181], index = distribution, name = "demand")
# the min prodcution is a fraction of the max
frac = 0.75


In [59]:
# MODEL
m = gp.Model("transportation")
x = m.addVars(production, distribution, name = "prod_ship")

can_produce = m.addConstrs((gp.quicksum(x[p,d] for d in distribution) <= max_prod[p] for p in production), name = "can_produce")
must_produce = m.addConstrs((gp.quicksum(x[p,d] for d in distribution) >= frac*max_prod[p] for p in production), name = "must_produce")
meet_demand = m.addConstrs((x.sum('*', d) >= n_demand[d] for d in distribution), name = "meet_demand")

m.setObjective(gp.quicksum(transp_cost[p,d]*x[p,d] for p in production for d in distribution), GRB.MINIMIZE)

In [ ]:
m.optimize()
x_values = pd.Series(m.getAttr("X", x), name='shipment', index=transp_cost.index)
sol = pd.concat([transp_cost, x_values], axis=1)
obj = m.getObjective().getValue()

print(f"Optimal solution: {obj}")
sol[sol.shipment > 0]

In [61]:
# MODEL 2
M = max_prod# max production
C = 30 # min ship

m = gp.Model("transportation1")
x = m.addVars(production, distribution, name = "prod_ship")
z = m.addVars(production, distribution, vtype=GRB.BINARY, name='ship_active')

can_produce = m.addConstrs((gp.quicksum(x[p,d] for d in distribution) <= max_prod[p] for p in production), name = "can_produce")
must_produce = m.addConstrs((gp.quicksum(x[p,d] for d in distribution) >= frac*max_prod[p] for p in production), name = "must_produce")
meet_demand = m.addConstrs((x.sum('*', d) >= n_demand[d] for d in distribution), name = "meet_demand")
# x <= 0 or x >= c  
# z1 or z2
# z1 -> x <= M*z1
# z2 -> x + M*z2 >= c  
# z1+z2 >= 1
# z1+z2 = 1 -> z1 = 1 - z2
# a --> x <= M*(1-z)
# b --> x + M*z >= c
min_ship_a = m.addConstrs((x[p,d] <= M[p]*(1-z[p,d]) for p in production for d in distribution), name='min_ship_a')
min_ship_b = m.addConstrs((x[p,d] + M[p]*z[p,d] >= C for p in production for d in distribution), name='min_ship_b')

m.setObjective(gp.quicksum(transp_cost[p,d]*x[p,d] for p in production for d in distribution), GRB.MINIMIZE)

 

In [ ]:

m.optimize()
x_values = pd.Series(m.getAttr("X", x), name='shipment', index=transp_cost.index)
sol = pd.concat([transp_cost, x_values], axis=1)
obj = m.getObjective().getValue()

print(f"Optimal solution: {obj}")
sol[sol.shipment > 0]

In [ ]:
###################################
## USANDO GLI INDICATOR CONSTRAINTS
###################################
m.addConstrs(((z[p,d] == 1) >> (x[p,d] >= C) for p in production for d in distribution), name='')
m.addConstrs(((z[p,d] == 0) >> (x[p,d] <= 0) for p in production for d in distribution), name='')

In [64]:
###################################
## SEMI CONTINUOUS DECISION VARIABLE 
###################################
x = m.addVars(production, distribution, vtype=GRB.SEMICONT, lb=C, name='prod_ship')


In [65]:
# ESEMPIO DUE - LIMITAZIONE FACILITIES

In [ ]:
import gurobipy as gp
import pandas as pd
from gurobipy import GRB


# DATA SETUP
path = 'https://raw.githubusercontent.com/Gurobi/modeling-examples/master/optimization101/Modeling_Session_1/'
transp_cost = pd.read_csv(path + 'cost.csv')
# get production and distribution locations from data frame
production = list(transp_cost['production'].unique())
distribution = list(transp_cost['distribution'].unique())
transp_cost = transp_cost.set_index(['production','distribution']).squeeze()

max_prod = pd.Series([180,200,140,80,180], index = production, name = "max_production")
n_demand = pd.Series([89,95,121,101,116,181], index = distribution, name = "demand")
# the min prodcution is a fraction of the max
frac = 0.75

max_prod2 = pd.Series([210,225,140,130,220], index = production, name = "max_production")

m = gp.Model("transportation")
x = m.addVars(production, distribution, name = "prod_ship")

can_produce = m.addConstrs((gp.quicksum(x[p,d] for d in distribution) <= max_prod2[p] for p in production), name = "can_produce")
meet_demand = m.addConstrs((x.sum('*', d) >= n_demand[d] for d in distribution), name = "meet_demand")

m.setObjective(gp.quicksum(transp_cost[p,d]*x[p,d] for p in production for d in distribution), GRB.MINIMIZE)

m.optimize()
x_values = pd.Series(m.getAttr('X', x), name = "shipment", index = transp_cost.index)
sol2 = pd.concat([transp_cost, x_values], axis=1)
sol2[sol2.shipment > 0]

In [ ]:
y = m.addVars(production, vtype=GRB.BINARY, name='activation') # indico se fabbrica è attiva o meno

# if sum(x[p,d]) <= 0 -> y[p] = 0
#  sum(x[p,d] for d in distribution) <= y[p]*M 
m.addConstrs((x.sum(p, '*') <= y[p]*max_prod2[p] for p in production), name='pp')


In [ ]:
# se una fabbrica a cherleston è aperta allora quella a cleveland e baltimore sono chiuse
reg_cond = m.addConstr((y['Cleveland']+y['Baltimore'] <=  (1-y['Charleston'])*2), name='pp')
m.update()
m.optimize()

In [ ]:
x_values = pd.Series(m.getAttr('x', x), name = "shipment", index = transp_cost.index)
sol2 = pd.concat([transp_cost, x_values], axis=1)
sol2[sol2.shipment > 0]